In [14]:
# 모델을 저장하기 위해 필요한 모듈을 설치한다.
!pip install pyyaml h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import os
import tensorflow as tf
from tensorflow import keras

In [16]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

train_y = train_y[:1000]
test_y = test_y[:1000]

train_x = train_x[:1000].reshape(-1, 28*28) / 255.0
test_x = test_x[:1000].reshape(-1, 28*28) / 255.0

In [17]:
# 현재의 목적은 모델을 저장하는 것을 테스트하는 것이 목적이기에 간단한 모델을 설계하도록 하겠다
def create_model():
    model = keras.Sequential([
        keras.layers.Dense(512, activation = 'elu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(optimizer = 'adam',
                  loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    
    return model

model = create_model()

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 훈련 중 체크포인트 저장
#### 재훈련할 필요없이 훈련된 모델을 사용하거나 훈련 프로세스가 중단된 경우 중단한 곳에서 훈련을 시작 할 수 있다.

In [18]:
# 체크포인트를 저장할 경로를 지정해준다.
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
# 모델의 가중치만 저장하도록 하겠다. 그럴경우 모델을 복원할 때는 동일한 아키텍쳐의 모델을 만든 다음 가중치를 설정해야된다.
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only = True,
                                                 verbose = 1)
model.fit(train_x,train_y,
          epochs = 10,
          validation_data = (test_x, test_y),
          callbacks=[cp_callback])

Epoch 1/10
18/32 [===============>..............] - ETA: 0s - loss: 1.3998 - sparse_categorical_accuracy: 0.5781
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 9ms/step - loss: 1.0600 - sparse_categorical_accuracy: 0.6890 - val_loss: 0.6564 - val_sparse_categorical_accuracy: 0.7820
Epoch 2/10
24/32 [=====================>........] - ETA: 0s - loss: 0.4181 - sparse_categorical_accuracy: 0.8802
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 0.3958 - sparse_categorical_accuracy: 0.8810 - val_loss: 0.5238 - val_sparse_categorical_accuracy: 0.8300
Epoch 3/10
22/32 [===================>..........] - ETA: 0s - loss: 0.3097 - sparse_categorical_accuracy: 0.9162
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 0.2967 - sparse_categorical_accuracy: 0.9200 - val_loss: 0.4868 - val_sparse_categorical_accuracy: 0.8420
Epoch 4/10
23/32 [==========

In [19]:
# 저장된 파일들을 확인해본다.
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'cp.ckpt.data-00000-of-00001', 'checkpoint']

In [20]:
# 새로운 모델을 만들어서 초기 상태 여부를 확인해본다.
model = create_model()

loss, acc = model.evaluate(test_x, test_y, verbose=1)

32/32 [==============================] - 0s 2ms/step - loss: 2.3617 - sparse_categorical_accuracy: 0.1100


In [21]:
# 체크포인트의 가중치를 로드해본다.
model.load_weights(checkpoint_path)

model.evaluate(test_x, test_y, verbose=1)

32/32 [==============================] - 0s 1ms/step - loss: 0.5047 - sparse_categorical_accuracy: 0.8390


[0.5047345757484436, 0.8389999866485596]

In [23]:
# 5에포크마다 가중치를 저장해보겠다.
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath= checkpoint_path,
    verbose = 1,
    save_weights_only = True,
    save_freq = 5*batch_size
)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_x, train_y,
          epochs=50,
          batch_size = batch_size,
          callbacks=[cp_callback],
          validation_data = (test_x, test_y),
          verbose = 0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [24]:
# 저장된 파일을 확인해본다.
os.listdir(checkpoint_dir)

['cp-0035.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0050.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0025.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index']

In [25]:
# 가장 최근에 저장된 체크포인트를 확인해본다.
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [27]:
# 새로 모델을 만들고 가중치를 로드해서 test데이터를 돌려본다.
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_x, test_y, verbose = 2)

32/32 - 0s - loss: 0.7737 - sparse_categorical_accuracy: 0.8400 - 136ms/epoch - 4ms/step


In [28]:
# 이번에는 모델 전체를 저장해보겠다.
model.save('saved_model/mymodel')

In [30]:
ls saved_model/mymodel

assets/  keras_metadata.pb  saved_model.pb  variables/


In [31]:
new_model = tf.keras.models.load_model('saved_model/mymodel')

new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [32]:
new_model.evaluate(test_x, test_y, verbose=2)

32/32 - 0s - loss: 0.7737 - sparse_categorical_accuracy: 0.8400 - 125ms/epoch - 4ms/step


[0.7736908197402954, 0.8399999737739563]

## HDF5 형식
 - keras는 HDF5 표준을 사용하여 기본저장 형식을 제공한다.

In [33]:
new_model.save('h5model/my_model.h5')

In [34]:
ls h5model

my_model.h5


In [38]:
new_model = tf.keras.models.load_model('h5model/my_model.h5')

new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
